In [99]:
# import libraries
import pandas as pd
pd.options.display.max_columns = 50
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# add import for Decision Treees
from sklearn.feature_selection import RFECV, SelectKBest, f_regression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
import pickle

### Step 1: Read in hold out data and best model

In [100]:
df = pd.read_csv('holdout_data.csv', index_col=0)
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23
5501,180000,2,2,1,44,0,0,0,0,0,0,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000
28857,130000,2,2,1,48,-2,-2,-2,-2,-2,-2,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849
11272,60000,2,1,1,43,-1,3,2,0,0,-1,495,330,495,330,165,340,0,330,0,0,340,0
8206,240000,1,1,1,42,0,0,0,0,0,0,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893
6362,100000,2,2,1,28,2,0,0,0,0,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0


In [101]:
final_model = pickle.load(open('final_model.pickle', 'rb'))

In [102]:
final_scaler = pickle.load(open('final_scaler.pickle', 'rb'))

### Step 2: Feature Engineering for holdout set

In [103]:
# making dataframe mor readable by creating new column names
rename_list = ['max_credit_given', 'gender', 'education', 'marital_status', 'age',
'pay_status_sept', 'pay_status_aug', 'pay_status_july', 'pay_status_june', 'pay_status_may', 'pay_status_april', 
'bill_sept', 'bill_aug', 'bill_july', 'bill_june', 'bill_may', 'bill_april',
'payment_sep', 'payment_aug', 'payments_jul', 'payment_jun', 'payment_may', 'payment_april', 'default']
col_rename = dict(zip(df.columns,rename_list))
df = df.rename(columns=col_rename)

In [104]:
# drop the row if has text in it
df.drop(df[df.gender == 'SEX'].index, inplace=True)

In [105]:
# changing all values from objects to
df = df.apply(pd.to_numeric, errors='coerce')

In [106]:
# fixing 'education' column by putting unknown variables in 'other' category (4)
fix_edu = (df.education == 5) | (df.education == 6) | (df.education == 0)
df.loc[fix_edu, 'education'] = 4

# fixing 'marital_status' column
df.loc[df.marital_status == 0, 'marital_status'] = 3

In [107]:
fix_status = (df.pay_status_sept == -2) | (df.pay_status_sept == 0)
df.loc[fix_status, 'pay_status_sept'] = -1

fix_status = (df.pay_status_aug == -2) | (df.pay_status_aug == 0)
df.loc[fix_status, 'pay_status_aug'] = -1

fix_status = (df.pay_status_july == -2) | (df.pay_status_july == 0)
df.loc[fix_status, 'pay_status_july'] = -1

fix_status = (df.pay_status_june == -2) | (df.pay_status_june == 0)
df.loc[fix_status, 'pay_status_june'] = -1

fix_status = (df.pay_status_may == -2) | (df.pay_status_may == 0)
df.loc[fix_status, 'pay_status_may'] = -1

fix_status = (df.pay_status_april == -2) | (df.pay_status_april == 0)
df.loc[fix_status, 'pay_status_april'] = -1

In [108]:
df[['gender','education','marital_status']] = df[['gender','education','marital_status']].astype('object')

# dummying
df = pd.get_dummies(df)
df.head()

,max_credit_given,age,pay_status_sept,pay_status_aug,pay_status_july,pay_status_june,pay_status_may,pay_status_april,bill_sept,bill_aug,bill_july,bill_june,bill_may,bill_april,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_april,gender_1,gender_2,education_1,education_2,education_3,education_4,marital_status_1,marital_status_2,marital_status_3
5501,180000,44,-1,-1,-1,-1,-1,-1,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000,0,1,0,1,0,0,1,0,0
28857,130000,48,-1,-1,-1,-1,-1,-1,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849,0,1,0,1,0,0,1,0,0
11272,60000,43,-1,3,2,-1,-1,-1,495,330,495,330,165,340,0,330,0,0,340,0,0,1,1,0,0,0,1,0,0
8206,240000,42,-1,-1,-1,-1,-1,-1,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893,1,0,1,0,0,0,1,0,0
6362,100000,28,2,-1,-1,-1,-1,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0,0,1,0,1,0,0,1,0,0


In [109]:
df.head()

,max_credit_given,age,pay_status_sept,pay_status_aug,pay_status_july,pay_status_june,pay_status_may,pay_status_april,bill_sept,bill_aug,bill_july,bill_june,bill_may,bill_april,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_april,gender_1,gender_2,education_1,education_2,education_3,education_4,marital_status_1,marital_status_2,marital_status_3
5501,180000,44,-1,-1,-1,-1,-1,-1,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000,0,1,0,1,0,0,1,0,0
28857,130000,48,-1,-1,-1,-1,-1,-1,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849,0,1,0,1,0,0,1,0,0
11272,60000,43,-1,3,2,-1,-1,-1,495,330,495,330,165,340,0,330,0,0,340,0,0,1,1,0,0,0,1,0,0
8206,240000,42,-1,-1,-1,-1,-1,-1,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893,1,0,1,0,0,0,1,0,0
6362,100000,28,2,-1,-1,-1,-1,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0,0,1,0,1,0,0,1,0,0


### Step 3: Predict the holdout set

In [110]:
# applying scaler
df = final_scaler.transform(df)

In [111]:
# features = df[['max_credit_given','gender','education',	'marital_status','age',	'pay_status_sept','pay_status_aug'	'pay_status_july','pay_status_june','pay_status_may','pay_status_april','bill_sept','bill_aug','bill_july','bill_june','bill_may','bill_april','payment_sep','payment_aug','payments_jul','payment_jun','payment_may','payment_april]]

In [112]:
# predicting on the cleaned up holdout dataframe

final_prediction = final_model.predict(df)

### Step 4: Export predictions as csv

In [113]:
final_prediction = pd.DataFrame(final_prediction, columns=['predictions'])

In [114]:
final_prediction.head()

,predictions
0,0
1,0
2,1
3,0
4,1


In [115]:
final_prediction.to_csv('predictions_sidney_kung.csv')